In [144]:
import numpy as np
import math

In [145]:
numberOfWordsInVocabulary = 61188
numberOfClasses = 20
numberOfTestingExamples = 7505

In [146]:
classProbabilityMatrix = np.zeros((numberOfClasses,))

In [147]:
wordProbabilityMatrix = np.zeros((numberOfWordsInVocabulary,numberOfClasses))

In [148]:
wordCountMatrix = np.zeros((numberOfWordsInVocabulary,numberOfClasses))

In [149]:
rowIndex = 1
docIdToLabel = {}
with open('data/train.label', 'r') as trainLabels:
    for line in trainLabels:
        label = int(line.rstrip('\n'))
        docIdToLabel[str(rowIndex)] = label
        classProbabilityMatrix[label-1] = classProbabilityMatrix[label-1] + 1
        rowIndex = rowIndex + 1
classProbabilityMatrix

array([ 480.,  581.,  572.,  587.,  575.,  592.,  582.,  592.,  596.,
        594.,  598.,  594.,  591.,  594.,  593.,  599.,  545.,  564.,
        464.,  376.])

In [150]:
totalTrainingExamples = classProbabilityMatrix.sum()
classProbabilityMatrix = np.log2(classProbabilityMatrix/totalTrainingExamples)
classProbabilityMatrix

array([-4.55318128, -4.27767752, -4.30020054, -4.26285518, -4.29265373,
       -4.25061851, -4.27519653, -4.25061851, -4.24090336, -4.24575276,
       -4.2360702 , -4.24575276, -4.25305756, -4.24575276, -4.24818358,
       -4.23365968, -4.36995946, -4.32052052, -4.60209088, -4.90548303])

In [151]:
with open('data/train.data','r') as trainData:
    for line in trainData:
        values = line.rstrip('\n').split(" ")
        label = docIdToLabel[values[0]]
        wordId = int(values[1])
        wordCount = int(values[2])
        matrixValue = wordCountMatrix[wordId-1,label-1]
        newValue = matrixValue + wordCount
        wordCountMatrix[wordId-1,label-1] = newValue
wordCountMatrix

array([[  13.,   60.,   11., ...,   10.,    0.,    0.],
       [  63.,   59.,   69., ...,  154.,   39.,   45.],
       [ 275.,    0.,    0., ...,    0.,    0.,    9.],
       ..., 
       [   0.,    0.,    0., ...,    0.,    0.,    0.],
       [   0.,    0.,    0., ...,    0.,    0.,    0.],
       [   0.,    0.,    0., ...,    0.,    0.,    0.]])

In [153]:
wordCountMatrix[:,1].sum()

110358.0

In [154]:
#beta = 1.0/numberOfWordsInVocabulary
beta = 1.0

In [155]:
vocabSize = numberOfWordsInVocabulary
for v in range(0,numberOfClasses):
    totalWordsInClassV = wordCountMatrix[:,v].sum()
    
    for w in range(0,numberOfWordsInVocabulary):
        wordId = int(w)
        label = int(v)
        countOfWInClassV = wordCountMatrix[w,v]
        probabilityOfWGivenV = ((countOfWInClassV) + beta)/ (totalWordsInClassV + beta*vocabSize)
        wordProbabilityMatrix[wordId,label] = probabilityOfWGivenV


In [156]:
wordProbabilityMatrix = np.log2(wordProbabilityMatrix)


In [157]:
testMatrix = np.zeros((numberOfTestingExamples,numberOfWordsInVocabulary))

In [158]:
rowIndex = 1
docIdToTestLabel = {}
with open('data/test.label', 'r') as testLabels:
    for line in testLabels:
        docIdToTestLabel[str(rowIndex)] = int(line.rstrip('\n'))
        #docIdToTestLabel.append(int(line.rstrip('\n')))
        rowIndex = rowIndex + 1

In [159]:
with open('data/test.data','r') as testData:   
    for line in testData:
        values = line.rstrip('\n').split(" ")
        label = docIdToTestLabel[values[0]]
        docId = int(values[0])
        wordId = int(values[1])
        wordCount = int(values[2])
        testMatrix[docId-1,wordId-1] = wordCount


In [160]:
classifySumMatrix = np.dot(testMatrix,wordProbabilityMatrix)

In [161]:
classifyProbabilityMatrix = classifySumMatrix + classProbabilityMatrix

In [162]:
prediction = np.argmax(classifyProbabilityMatrix[7503,:]) + 1

In [163]:
prediction

16

In [164]:
#file = open("classify2.txt", "w")
errorCount = 0
for e in range(0,numberOfTestingExamples):
    prediction = np.argmax(classifyProbabilityMatrix[e,:]) + 1
    if prediction != docIdToTestLabel[str(e + 1)]:
        errorCount = errorCount + 1
        
print(errorCount)

1643


In [165]:
1.0 - errorCount/numberOfTestingExamples

0.7810792804796802